In [501]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import periodogram
%matplotlib inline

# Machine Intelligence II

## Exercise Sheet 06: Independent Component Analysis

### Group WALB: Wichert, Alevi, Lang, Boelts

### 6.1

In [508]:
# (a)
rate = 8192
p = 18000
seed = 13
sound1 = np.loadtxt('../sounds/sound1.dat')
sound2 = np.loadtxt('../sounds/sound2.dat')
sounds = np.concatenate([[sound1, sound2]], axis=1)
wavfile.write('original1.wav', rate, sound1)
wavfile.write('original2.wav', rate, sound2)

(2, 18000)


In [ ]:
# (b)
A = np.linalg.inv(np.random.RandomState(seed).rand(2,2))
X = A.dot(sounds)

In [ ]:
# (c)
dist = X # for 6.3 (a) (ii)
X = X[:,np.random.RandomState(seed+1).permutation(X.shape[1])]
dist_perm = X # for 6.3 (a) (ii)

In [ ]:
# (d)
for i in range(2):
    for j in range(2):
        corr = np.cov(sounds[i,:], X[j,:]) / sounds[i,:].var() / X[j,:].var()
        print('correlation matrix between s_{} and x_{}:\n {}\n'.format(i, j, corr))

In [ ]:
# (e)
X = X - X.mean(axis=1).reshape(2,1)

In [ ]:
# (f)
W = np.linalg.inv(np.random.RandomState(seed+1).rand(2,2))
W_nat = np.linalg.inv(np.random.RandomState(seed+1).rand(2,2))

### 6.2

In [ ]:
def f(y):
    return 1 / (1 + np.exp(-y))

def phi(y):
    return 1 - 2 * f(y)

In [ ]:
# (a)
def update_regular(W, x, eta):
    W_inv = np.linalg.inv(W)    
    delta_W = W_inv.T + phi(W.dot(x)).reshape(2,1).dot(x.reshape(1,2))
    return W + eta * delta_W,  delta_W, eta * delta_W

In [ ]:
# (b)
def update_natural(W, x, eta):
    phee = phi(W.dot(x)).reshape(2,1)
    delta_W = np.dot(phee.dot(np.dot(W, x).reshape(1,2)), W)
    delta_W = delta_W + W # multiplied out delta function
    delta_W[0,0] = 0 # Bell-Seijnowski solution
    delta_W[1,1] = 0 # Bell-Seijnowski solution
    return W + eta * delta_W,  delta_W, eta * delta_W

In [ ]:
# (c)
eta_0 = 20
delta_W_norms = []
delta_W_norms_eta = []
for t in range(X.shape[1]):
    eta = eta_0 / (t + 1)
    x = X[:,t]
    W, delta_W, delta_W_eta = update_regular(W, x, eta)
    if t % 1000 == 0:
        delta_W_norms.append(np.sum(delta_W ** 2)) # for 6.3 (c)
        delta_W_norms_eta.append(np.sum(delta_W_eta ** 2)) # for 6.3 (c)
    
rec = W.dot(A.dot(sounds))

eta_0 = .15
delta_W_norms_nat = []
delta_W_norms_nat_eta = []
W_nat = np.linalg.inv(np.random.RandomState(seed+2).rand(2,2))
W_nat[0,0] = 1 # Bell-Seijnowski solution
W_nat[1,1] = 1 # Bell-Seijnowski solution
for t in range(X.shape[1]):
    eta = eta_0 / (t + 1)
    x = X[:,t]
    W_nat, delta_W_nat, delta_W_nat_eta = update_natural(W_nat, x, eta)
    assert not np.isnan(W_nat[0,0])
    if t % 1000 == 0:
        delta_W_norms_nat.append(np.sum(delta_W_nat ** 2)) # for 6.3 (c)
        delta_W_norms_nat_eta.append(np.sum(delta_W_nat_eta ** 2)) # for 6.3 (c)

rec_nat = W_nat.dot(A.dot(sounds))

### 6.3

In [ ]:
# (a)

wavfile.write('rec1.wav', rate, rec[0,:])
wavfile.write('rec2.wav', rate, rec[1,:])
wavfile.write('rec_nat1.wav', rate, rec_nat[0,:])
wavfile.write('rec_nat2.wav', rate, rec_nat[1,:])
wavfile.write('dist1.wav', rate, dist[0,:]) 
wavfile.write('dist2.wav', rate, dist[1,:])
wavfile.write('dist_perm1.wav', rate, dist_perm[0,:])
wavfile.write('dist_perm2.wav', rate, dist_perm[1,:])

In [ ]:
time = np.arange(p) / rate

plt.figure(figsize=(12,4))
plt.subplot('121')
plt.plot(time, sound1)
plt.xlabel('time [s]')
plt.ylabel('amplitude')
plt.title('original signal 1 (birds chirping)')
plt.subplot('122')
plt.plot(time, sound2)
plt.xlabel('time [s]')
plt.ylabel('amplitude')
plt.title('original signal 2 (halleluja)')

plt.figure(figsize=(12,4))
plt.subplot('121')
plt.plot(time, rec[0,:])
plt.xlabel('time [s]')
plt.ylabel('amplitude')
plt.title('first recovered signal (regular update rule)')
plt.subplot('122')
plt.plot(time, rec[1,:])
plt.xlabel('time [s]')
plt.ylabel('amplitude')
plt.title('second recovered signal (regular update rule)')

plt.figure(figsize=(12,4))
plt.subplot('121')
plt.plot(time, rec_nat[0,:])
plt.xlabel('time [s]')
plt.ylabel('amplitude')
plt.title('first recovered signal (natural update rule)')
plt.subplot('122')
plt.plot(time, rec_nat[1,:])
plt.xlabel('time [s]')
plt.ylabel('amplitude')
plt.title('second recovered signal (natural update rule)')

plt.figure(figsize=(12,4))
plt.subplot('121')
plt.plot(time, dist[0,:])
plt.xlabel('time [s]')
plt.ylabel('amplitude')
plt.title('mixed sources before permutation')
plt.subplot('122')
plt.plot(time, dist[1,:])
plt.xlabel('time [s]')
plt.ylabel('amplitude')
plt.title('mixed sources before permutation')

plt.figure(figsize=(12,4))
plt.subplot('121')
plt.plot(time, dist_perm[0,:])
plt.xlabel('time [s]')
plt.ylabel('amplitude')
plt.title('mixed sources after permutation')
plt.subplot('122')
plt.plot(time, dist_perm[1,:])
plt.xlabel('time [s]')
plt.ylabel('amplitude')
plt.title('mixed sources after permutation');

In [ ]:
# (b)

for i in range(2):
    for j in range(2):
        corr = np.cov(sounds[i,:], rec[j,:]) / sounds[i,:].var() / rec[j,:].var()
        print('correlation matrix between s_{} and rec_{}:\n {}\n'.format(i, j, corr))

In [ ]:
# (c)

plt.plot(delta_W_norms)
plt.title(r'convergence of $\Delta W$ (regular, without learning rate)')
plt.xlabel('update # [k]')
plt.ylabel(r'$||\Delta W||^2_F$')

plt.figure()
plt.plot(delta_W_norms_nat)
plt.title(r'convergence of $\Delta W$ (natural, without learning rate)')
plt.xlabel('update # [k]')
plt.ylabel(r'$||\Delta W||^2_F$')

plt.figure()
plt.plot(delta_W_norms_eta)
plt.title(r'convergence of $\Delta W$ (regular, with learning rate)')
plt.xlabel('update # [k]')
plt.ylabel(r'$||\Delta W||^2_F$')

plt.figure()
plt.plot(delta_W_norms_nat_eta)
plt.title(r'convergence of $\Delta W$ (natural, with learning rate)')
plt.xlabel('update # [k]')
plt.ylabel(r'$||\Delta W||^2_F$');

In [ ]:
# (c) (continued)

def whiten_data(data):
    C = np.cov(data)
    w, V = np.linalg.eigh(C)
    D = np.diag(1 / np.sqrt(w))
    M = D.dot(V)
    return M.dot(data)
    
X_white = whiten_data(X)

eta_0 = 20
delta_W_norms = []
delta_W_norms_eta = []
for t in range(X_white.shape[1]):
    eta = eta_0 / (t + 1)
    x = X_white[:,t]
    W, delta_W, delta_W_eta = update_regular(W, x, eta)
    if t % 1000 == 0:
        delta_W_norms.append(np.sum(delta_W ** 2)) # for 6.3 (c)
        delta_W_norms_eta.append(np.sum(delta_W_eta ** 2)) # for 6.3 (c)
    
rec = W.dot(A.dot(sounds))

eta_0 = .15
delta_W_norms_nat = []
delta_W_norms_nat_eta = []
W_nat = np.linalg.inv(np.random.random(size=(2,2)))
W_nat[0,0] = 1 # Bell-Seijnowski solution
W_nat[1,1] = 1 # Bell-Seijnowski solution
for t in range(X_white.shape[1]):
    eta = eta_0 / (t + 1)
    x = X_white[:,t]
    W_nat, delta_W_nat, delta_W_nat_eta = update_natural(W_nat, x, eta)
    assert not np.isnan(W_nat[0,0])
    if t % 1000 == 0:
        delta_W_norms_nat.append(np.sum(delta_W_nat ** 2)) # for 6.3 (c)
        delta_W_norms_nat_eta.append(np.sum(delta_W_nat_eta ** 2)) # for 6.3 (c)

rec_nat = W_nat.dot(A.dot(sounds))

plt.plot(delta_W_norms)
plt.title(r'convergence of $\Delta W$ (regular, without learning rate)')
plt.xlabel('update # [k]')
plt.ylabel(r'$||\Delta W||^2_F$')

plt.figure()
plt.plot(delta_W_norms_nat)
plt.title(r'convergence of $\Delta W$ (natural, without learning rate)')
plt.xlabel('update # [k]')
plt.ylabel(r'$||\Delta W||^2_F$')

plt.figure()
plt.plot(delta_W_norms_eta)
plt.title(r'convergence of $\Delta W$ (regular, with learning rate)')
plt.xlabel('update # [k]')
plt.ylabel(r'$||\Delta W||^2_F$')

plt.figure()
plt.plot(delta_W_norms_nat_eta)
plt.title(r'convergence of $\Delta W$ (natural, with learning rate)')
plt.xlabel('update # [k]')
plt.ylabel(r'$||\Delta W||^2_F$');

In [ ]:
# (d)

def plot_density(signal1, signal2, name):
    f1, PXX_den1 = periodogram(signal1, rate)
    f2, PXX_den2 = periodogram(signal2, rate)
    plt.figure(figsize=(12,4))
    plt.subplot('121')
    plt.plot(f1, PXX_den1)
    plt.title('power density estimate of {} 1'.format(name))
    plt.xlabel('frequency [Hz]')
    plt.ylabel('density')
    plt.subplot('122')
    plt.plot(f2, PXX_den2)
    plt.title('power density estimate of {} 2'.format(name))
    plt.xlabel('frequency [Hz]')
    plt.ylabel('density')
    
plot_density(sound1, sound2, 'true signal')
plot_density(dist[0,:], dist[1,:], 'mixed signal')
plot_density(rec[0,:], rec[1,:], 'unmixed signal (regular)')
plot_density(rec_nat[0,:], rec_nat[1,:], 'unmixed signal (natural)')